In [1]:
import os
import xarray as xr
import pandas as pd
import datetime as dt
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt

# Introduction

Make river input scenarios for OF800

Questions (maybe for Miljødirektoratet) & future improvement needs:
* Currently don't do any scenarios for Sweden. Should we do something simple there?
* Andre used the wrong area-scaling of discharge. Therefore baseline inputs are also wrong.

# User input

In [2]:
# User input
# Version number
v = 3  # For file naming

# Filepath to baseline daily river data
fpath_baseline_data = r"/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/martini_rivers_v9_1990_2022_stage1data.nc"

# Filepath to river metadata csvs
real_riv_metadata = r"../data/real_river_metadata.csv"

# TEOTIL results (annual source apportionment)
# Baseline and scenario data
teotil_res_csv_fpath = (
    r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios/teotil3_oslomod_results_2017-2019.csv"
)

# Start and end year to summarise TEOTIL data over (inclusive??)
start_year = 2017
end_year = 2019

# Folders for output files
netcdf_outfolder = r"/home/jovyan/shared/common/oslofjord_modelling/MARTINI800v10_river_inputs/river_scenarios"
summary_csv_folder = r"/home/jovyan/shared/common/oslofjord_modelling/phase3_scenarios/summary_csvs"

# REAL river numbers to use
# full OF800 domain: model rivs 5-29 incl. Drop Sweden, becomes 7 to 29 (incl.). Real rivs 4 to 23.
# (v3: scenarios generated for vassdragsområder 001 (real riv 6-Tista, model riv 7) to 017 (real riv 23-Kammerfosselva, model riv 29), i.e. full OF800 domain excl. sweden)
oslofjord_riv_nos = range(6, 24)  # Whole OF domain, excluding 2 Swedish rivers
print(f"Generating scenarios for real rivers:\n{list(oslofjord_riv_nos)}")

# Species to alter in the scenarios (teotil3 species; mapping to of800 species done below)
# choose from ['din', 'tdp', 'ss', 'toc', 'ton', 'totn', 'totp', 'tpp']
t3_par_list = ['din', 'tdp', 'toc', 'ton', 'totn', 'totp', 'tpp', 'ss']

Generating scenarios for real rivers:
[6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]


# Set up & read in river metadata

In [3]:
# Dictionary to map from TEOTIL3 variable name to OF800 variable names
# We assume the same % reduction across all of800 variable names returned for a given
# TEOTIL3 variable name
# N.B. river_TIP is not used downstream in OF800, so do not include in the mapping.
var_mapping_teotil3_to_of800 = {
    'din': ['river_NH4N', 'river_NO3NO2N'],
    'ton': ['river_DON', 'river_PON'],
    'totn': ['river_TOTN'],
    'totp': ['river_TOTP'],
    'tdp': ['river_SRP', 'river_DOP'],
    'tpp': ['river_POP'],  # PIP isn't used in Martini, so don't worry about
    'toc': ['river_DOC', 'river_POC'],
    #'ss': ['river_SPM']#  # SPM is in a separate netcdf file. Just provide Phil with factors
}

# Calculate start date (inclusive), end date (day after last day)
start_date = dt.datetime(start_year, 1, 1)
end_date = dt.datetime(end_year, 12, 31)
end_date += pd.Timedelta(days=1)

# River chemistry metadata
river_meta_df = pd.read_csv(real_riv_metadata, index_col=0, dtype={'Vassom':str})
# Limit to just Oslofjord rivers
river_meta_df = river_meta_df[river_meta_df.index.isin(oslofjord_riv_nos)]
# Add 'total' row for use later
river_meta_df.loc['Total', ['river_name', 'Regine', 'Regine_to_sea', 'Vassom']] = 'Total'
river_meta_df.query('real_river in @oslofjord_riv_nos')

,river_name,Outflow_lat,Outflow_lon,Regine,Regine_to_sea,Vassom,Vassom_area_land,Vassom_area_tot,Andre_MCA_area,Andre_area_q,Overestimate (%),Comment
real_river,,,,,,,,,,,,
6,Tista,59.119,11.370,001.A1,001.A1,001.,2495.0,2507.0,2507.0,1584.0,0.0,NaN
7,Glomma,59.206,10.953,002.A51,002.A11,002.,42446.0,43116.0,43116.0,41967.0,2.0,Monitoring point upstream of Sarpsborg RA. So ...
8,Mosseelva,59.439,10.662,003.A1,003.A1,003.,854.0,1052.0,1054.0,694.0,23.0,NaN
9,Hølenelva,59.523,10.690,004.A0,004.A0,004.,204.0,227.0,NaN,NaN,11.0,NaN
10,Årungen,59.720,10.728,005.3A,005.3A,005.,280.0,368.0,144.0,85.0,31.0,NaN
11,Akerselva,59.908,10.756,006.A10,006.A10,006.,392.0,415.0,415.0,307.0,6.0,NaN
12,Lysakerelva,59.914,10.640,007.A0,007.A0,007.,202.0,211.0,211.0,177.0,4.0,NaN
13,Sandvikselva,59.890,10.523,008.A11,008.A11,008.,279.0,311.0,311.0,226.0,11.0,NaN
14,Åros,59.704,10.519,009.A0,009.A0,009.,215.0,253.0,NaN,NaN,18.0,NaN


# TEOTIL source apportionment

## Read in data

TEOTIL columns:
* 'year',
* 'regine',
* 'regine_down',
* 'accum_agriculture-background_din_kg',
* 'accum_agriculture-background_ss_kg',
* 'accum_agriculture-background_tdp_kg',
* 'accum_agriculture-background_toc_kg',
* 'accum_agriculture-background_ton_kg',
* 'accum_agriculture-background_totn_kg',
* 'accum_agriculture-background_totp_kg',
* 'accum_agriculture-background_tpp_kg',
* ... repeat for all the other sources, e.g. instead of 'agriculture-background', have 'urban', 'wood', etc.
* ... repeat with local_ instead of accum_ for local inputs to the regine, rather than accumulated upstream inputs. Here, we're interested in accumulated upstream inpust.

i.e. param choices are totn, din, ton, ss, totp, tdp, tpp, toc

In [4]:
def extract_source(col_name):
    """
    Function to rename teotil columns
    """
    parts = col_name.split('_')
    return parts[1]


def read_teotil_data(teotil_data_fpath, par):
    """
    Function to read raw TEOTIL data, pick the columns of interest (just accumulated inputs
    from all upstream areas), rename columns, truncate to the period of interest.

    Do this for areas upstream of a given regine of interest
    """
    df = pd.read_csv(teotil_data_fpath)

    # Just pick out accumulated inputs (from all upstream areas) for the outflow reginer of interest
    # Also just pick columns for the single parameter of interest
    cols = ["scenario", "regine", "year"] + [
        col for col in df.columns if f"_{par}_" in col and col.startswith("accum")
    ]
    df = df.loc[df['regine'].isin(river_meta_df['Regine'])][cols]

    # Truncate to start of model period
    df = df.loc[df['year'] >= start_year]
    df = df.loc[df['year'] <= end_year]

    # Replace 'regine' index with river numbers, for easier matching to the river data
    mapping_dict = river_meta_df.reset_index().set_index('Regine')['real_river'].to_dict()
    df.index = df['regine'].map(mapping_dict)
    df.index.name = 'real_river'
    df = df.drop('regine', axis=1)

    # Rename columns
    cols_to_exclude = ['scenario', 'year']
    new_col_names_dict = {col: extract_source(col) for col in df.columns if col not in cols_to_exclude}
    new_col_names = cols_to_exclude + list(new_col_names_dict.values())
    col_dict = dict(zip(df.columns, new_col_names))
    df = df.rename(columns=col_dict)

    return df


def teotil_average_over_years(df):
    """
    Average over all years to get a single value per main catchment area (or real river).
    Replace 'regine' index with river numbers
    """

    # Average over the year column per regine
    group_col = df.index.name
    av_df = df.reset_index().groupby(group_col).mean().drop('year', axis=1)

    return av_df



In [5]:
# Read baseline and scenario TEOTIL data (units kg/yr) and average over time
teotil_av_df_dict = {}  # key: (scenario, par), returns df of time-averaged loads with real_river index, one column per source

# Make list of scenarios, for looping later
scenario_list = []

for i, par in enumerate(t3_par_list):
    tmp_teotil_df = read_teotil_data(teotil_res_csv_fpath, par)

    unique_scenarios = tmp_teotil_df['scenario'].unique()

    for scenario in unique_scenarios:
        # Select rows where the 'scenario' column matches the current scenario
        tmp_scenario_df = tmp_teotil_df[tmp_teotil_df['scenario'] == scenario]
        tmp_scenario_df = tmp_scenario_df.drop('scenario', axis=1)

        tmp_teotil_av_df = teotil_average_over_years(tmp_scenario_df)
        teotil_av_df_dict[(scenario, par)] = tmp_teotil_av_df

        # Populate scenario list (just once)
        if i == 0:
            scenario_list.append(scenario)

        # Print for QC
        if par == 'ss':
            print(f'------------Scenario: {scenario} ---------')
            print(tmp_scenario_df.loc[7, :].drop('year', axis=1).sum(axis=1) / 1000)  # tonnes/yr for Glomma

# Drop baseline from scenario_list for future use
scenario_list.remove('Baseline')
print(scenario_list)

------------Scenario: Baseline ---------
real_river
7    145049.064101
7    164304.529149
7    163799.740292
dtype: float64
------------Scenario: Scenario_A ---------
real_river
7    53335.899516
7    59751.210070
7    59569.803264
dtype: float64
------------Scenario: Scenario_B ---------
real_river
7    38283.400987
7    42433.031222
7    42310.896925
dtype: float64
['Scenario_A', 'Scenario_B']


## Load reductions over whole OF (just for river inputs)

In [6]:
# Work out overall load reductions
tot_dict = {}
for par in t3_par_list:
    tot_li = []
    for scenario in ['Baseline'] + scenario_list:
        of_total_s = teotil_av_df_dict[(scenario, par)].copy()
        of_total_s = of_total_s.sum(axis=1) / 1000
        of_total_s.name = scenario
        tot_li.append(of_total_s)

    tot_df = pd.concat(tot_li, axis=1)
    tot_df.loc['Whole OF', :] = tot_df.sum(axis=0)

    tot_df['f_red_A'] = tot_df['Scenario_A'] / tot_df['Baseline']
    tot_df['f_red_B'] = tot_df['Scenario_B'] / tot_df['Baseline']
    tot_df['Percent_reduction_A'] = (100 * (1 - tot_df['f_red_A'])).round(2)
    tot_df['Percent_reduction_B'] = (100 * (1 - tot_df['f_red_B'])).round(2)

    tot_dict[par] = tot_df

tot_dict['din']

,Baseline,Scenario_A,Scenario_B,f_red_A,f_red_B,Percent_reduction_A,Percent_reduction_B
real_river,,,,,,,
6,371.447101,359.390419,275.721181,0.967541,0.742289,3.25,25.77
7,9543.466110,8647.859550,6857.853595,0.906155,0.718591,9.38,28.14
8,235.297826,231.286863,180.483841,0.982954,0.767044,1.70,23.30
9,175.334598,170.386060,129.379245,0.971777,0.737899,2.82,26.21
10,61.799531,60.296928,47.048101,0.975686,0.761302,2.43,23.87
11,36.979296,36.830038,35.069431,0.995964,0.948353,0.40,5.16
12,33.832345,33.475422,30.813888,0.989450,0.910782,1.05,8.92
13,69.879051,68.809331,61.270822,0.984692,0.876812,1.53,12.32
14,69.772272,67.610049,54.277904,0.969010,0.777929,3.10,22.21


In [22]:
# Quick manual check

riv_no = 7
par = 'din'

test = teotil_av_df_dict[('Baseline', par)].copy().loc[riv_no, :]/1000  # tonnes/yr
print(f"Baseline: {test.sum().round(2)}")

test = teotil_av_df_dict[('Scenario_A', par)].copy().loc[riv_no, :]/1000  # tonnes/yr
print(f"Scenario_A: {test.sum().round(2)}")

test = teotil_av_df_dict[('Scenario_B', par)].copy().loc[riv_no, :]/1000  # tonnes/yr
print(f"Scenario_B: {test.sum().round(2)}")

Baseline: 9543.47
Scenario_A: 8647.86
Scenario_B: 6857.85


## TEOTIL proportion source apportionment per river

Calculate the proportion each sector makes to the total, averaged over all the years of interest, for each parameter

In [7]:
f_sector_df_dict = {}  # key: par, returns baseline contribution per sector

for par in t3_par_list:

    # Total accumulated input per regine from all sources
    bsl_teotil_av_df = teotil_av_df_dict[('Baseline', par)].copy()
    teotil_total_s = bsl_teotil_av_df.sum(axis=1)

    # Proportion per regine and source (checked that rows sum to 1, they do)
    tmp_f_sector_df = bsl_teotil_av_df.divide(teotil_total_s, axis=0)
    # print(f_sector_df.sum(axis=1).tail())

    # Remove any rows where real_river=NaN, i.e. Sweden (if present)
    tmp_f_sector_df = tmp_f_sector_df.reset_index().dropna(subset=['real_river']).set_index('real_river')

    # Select just the rivers of interest in the current domain
    tmp_f_sector_df = tmp_f_sector_df[tmp_f_sector_df.index.isin(oslofjord_riv_nos)]

    # # Also calculate total flux over all rivers
    # # Add as another row to f_sector_df with index='Total'
    # total_flux = bsl_teotil_av_df.sum().sum()
    # tmp_f_sector_df.loc['Total'] = bsl_teotil_av_df.sum(axis=0) / total_flux

    # print(par)
    # print((tmp_f_sector_df.loc['Total', :] * 100).round(1))
    # print("--------------------")

    f_sector_df_dict[par] = tmp_f_sector_df

In [11]:
# Quick manual checks
bsl_teotil_av_df = teotil_av_df_dict[('Baseline', 'din')].copy()
teotil_total_s = bsl_teotil_av_df.sum(axis=1)
print(teotil_total_s.loc[7] / 1000)

9543.46611033241


In [12]:
bsl_teotil_av_df.loc[7, :] / 1000

agriculture-background     666.280612
agriculture               5996.525510
aquaculture                  0.000000
industry                     3.508970
lake                       278.626902
large-wastewater          1436.192249
spredt                     243.509493
upland                     204.767914
urban                      268.496179
wood                       445.558280
Name: 7, dtype: float64

In [14]:
f_sector_df_dict['din'].loc[7, :]

agriculture-background    0.069815
agriculture               0.628338
aquaculture               0.000000
industry                  0.000368
lake                      0.029196
large-wastewater          0.150490
spredt                    0.025516
upland                    0.021456
urban                     0.028134
wood                      0.046687
Name: 7, dtype: float64

# Calculate scenario reduction factors

Both WWTW and agricultural scenarios are now provided by TEOTIL (in contrast to work in 2024). There are two scenarios, one is more moderate, one more optimistic.

In [15]:
# Calculate factor reductions per sector, scenario and real river

f_red_df_dict = {}  # key: (scenario, teotil par). Returns: df, real_river index, one col per source, scen/bsl

# List of sectors where change factors != 1
sector_cols = []

for scenario in scenario_list:
    for par in t3_par_list:
        f_red_df = teotil_av_df_dict[(scenario, par)] / teotil_av_df_dict[('Baseline', par)]

        # Have some NaNs (when baseline = 0). Replace any NaNs with 1 (i.e. no change)
        f_red_df.replace([np.inf, -np.inf], 1, inplace=True)
        f_red_df.fillna(1, inplace=True)

        f_red_df_dict[(scenario, par)] = f_red_df

        # Populate change_sectors list
        changed_cols = [col for col in f_red_df.columns if (f_red_df[col] != 1).any()]
        sector_cols.append(changed_cols)

# Make the sector_cols list
# Flatten the list
sector_cols = [item for sublist in sector_cols for item in sublist]
# Drop duplicates
sector_cols = list(set(sector_cols))
print("Sectors updated in the scenarios for at least one param:")
print(sector_cols)

Sectors updated in the scenarios for at least one param:
['agriculture-background', 'agriculture', 'large-wastewater']


In [16]:
f_red_df_dict[('Scenario_A', 'din')].head()

,agriculture-background,agriculture,aquaculture,industry,lake,large-wastewater,spredt,upland,urban,wood
real_river,,,,,,,,,,
6,0.980161,0.980161,1.0,1.0,1.0,0.658689,1.0,1.0,1.0,1.0
7,0.986353,0.986353,1.0,1.0,1.0,0.439712,1.0,1.0,1.0,1.0
8,0.979065,0.979065,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0
9,0.968437,0.968437,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0
10,0.973801,0.973801,1.0,1.0,1.0,1.000000,1.0,1.0,1.0,1.0


In [17]:
# Calculate overall reduction factor, averaged over sector (taking proportion
# contribution per sector into account)

# Reduction factors are multiplicative, i.e. new conc = baseline conc * factor

# Also make scenario summary table for report and Andre

rename_f_sector_dict = {'agriculture': 'f_jordbruk',
                        'large-wastewater': 'f_ra',
                        'agriculture-background': 'f_jordbruk-bakgrunn'}

rename_f_red_dict = {'agriculture': 'f-red_jordbruk',
                     'large-wastewater': 'f-red_ra',
                     'agriculture-background': 'f-red_jordbruk-bakgrunn'}

scen_summary_df_dict = {}  # key: (scenario, par). Includes total reduction factor

for scenario in scenario_list:
    for par in t3_par_list:

        # Baseline proportion of total load from the sectors of interest
        scen_summary_df = f_sector_df_dict[par][sector_cols].copy()
        scen_summary_df = scen_summary_df.rename(rename_f_sector_dict, axis=1)

        # Reduction factors for the sectors of interest
        f_red_df = f_red_df_dict[(scenario, par)][sector_cols].copy()
        f_red_df = f_red_df.rename(rename_f_red_dict, axis=1)

        scen_summary_df = scen_summary_df.join(f_red_df)

        # Calculate f-red_total
        # N.B. this is long-winded. TODO: replace with calc directly from total loads in
        # scenarios and baseline. But below is good for validation. Get same result.
        # Dynamically calculate f-red_total the long-winded way, for validation of all component parts
        included_p_sum = 0
        f_red_total = 0

        for sector in sector_cols:
            p_i = scen_summary_df[rename_f_sector_dict[sector]]
            f_i = scen_summary_df[rename_f_red_dict[sector]]
            f_red_total += p_i * f_i
            included_p_sum += p_i

        # Add excluded sectors with f_i = 1
        remaining_p_sum = 1 - included_p_sum
        f_red_total += remaining_p_sum

        scen_summary_df['f-red_total'] = f_red_total

        # Add in total load per river, to calculate new totals for whole OF
        total_load_series = teotil_av_df_dict[(scenario, par)].sum(axis=1) / 1000
        scen_summary_df["Load_T-per-yr"] = scen_summary_df.index.map(total_load_series)

        # Add explicit percent reduction column, for clarity
        scen_summary_df['Percent reduction'] = (100 * (1 - scen_summary_df['f-red_total'])).round(1)

        # Add river name
        scen_summary_df['river_name'] = scen_summary_df.index.map(river_meta_df['river_name'])

        # Save for calculating new daily concentrations
        scen_summary_df_dict[(scenario, par)] = scen_summary_df

    # ------------ Make single long df-----------------
    dfs_list = []

    for key, df in scen_summary_df_dict.items():
        df['Teotil3 parameter'] = key[1]
        dfs_list.append(df)

    scen_summary_df = pd.concat(dfs_list, ignore_index=False).reset_index()

    # Reorder cols
    first_cols = ['real_river', 'river_name', 'Teotil3 parameter']
    scen_summary_df = scen_summary_df[first_cols + [col for col in scen_summary_df.columns if col not in first_cols]]

    # Save to csv
    fpath = os.path.join(summary_csv_folder, f"reduction_factors_per_river_scenario-{scenario}.csv")
    scen_summary_df.to_csv(fpath, index=False)

In [21]:
scen_summary_df_dict[('Scenario_A', 'din')]

,f_jordbruk-bakgrunn,f_jordbruk,f_ra,f-red_jordbruk-bakgrunn,f-red_jordbruk,f-red_ra,f-red_total,Load_T-per-yr,Percent reduction,river_name,Teotil3 parameter
real_river,,,,,,,,,,,
6,0.069869,0.628824,0.054487,0.980161,0.980161,0.658689,0.967541,359.390419,3.2,Tista,din
7,0.069815,0.628338,0.150490,0.986353,0.986353,0.439712,0.906155,8647.859550,9.4,Glomma,din
8,0.081426,0.732836,0.014471,0.979065,0.979065,1.000000,0.982954,231.286863,1.7,Mosseelva,din
9,0.089419,0.804771,0.012188,0.968437,0.968437,1.000000,0.971777,170.386060,2.8,Hølenelva,din
10,0.092807,0.835261,0.001230,0.973801,0.973801,1.000000,0.975686,60.296928,2.4,Årungen,din
11,0.019352,0.174172,0.000410,0.979143,0.979143,1.000000,0.995964,36.830038,0.4,Akerselva,din
12,0.039443,0.354987,0.000329,0.973253,0.973253,1.000000,0.989450,33.475422,1.1,Lysakerelva,din
13,0.052942,0.476481,0.005804,0.971085,0.971085,1.000000,0.984692,68.809331,1.5,Sandvikselva,din
14,0.074366,0.669295,0.000000,0.958328,0.958328,1.000000,0.969010,67.610049,3.1,Åros,din


# Calculate new daily concentrations

## Read in daily river data netcdf & tidy

In [ ]:
# If present, drop 'ss' from t3_par_li
if 'ss' in t3_par_list:
    t3_par_li_no_ss = t3_par_list.copy()
    t3_par_li_no_ss.remove('ss')

# Make list of of800 chem variables to be updated (for slicing baseline netcdf)
of800_chem_var_li = []
for par in t3_par_li_no_ss:
    of800_chem_var_li.extend(var_mapping_teotil3_to_of800[par])

bsl_ds = xr.open_dataset(fpath_baseline_data)

# Convert to dataframe for easier manipulation (by me!)
bsl_conc_df = bsl_ds.to_dataframe().loc[:, of800_chem_var_li]

# Convert all numeric columns to float64 (chem cols were float32)
bsl_conc_df[of800_chem_var_li] = bsl_conc_df[of800_chem_var_li].astype(float)

bsl_conc_df = bsl_conc_df.reset_index()  # Drop multiindex

# Select just the Oslofjord rivers
bsl_conc_df = bsl_conc_df[bsl_conc_df['real_river'].isin(oslofjord_riv_nos)]

# Truncate to start and end date
bsl_conc_df = bsl_conc_df.query('@start_date <= river_time <= @end_date')

bsl_conc_df.tail()

In [ ]:
# # Explore for specific rivers/times
# temp = conc_df.loc[conc_df['real_river'] == 14, ['river_time', 'river_NO3NO2N']]
# temp

## Calculate scenario concentrations

In [ ]:
scen_conc_dict = {}  # key: scenario, returns df of scenario concentration data for of800 vars

riv = 7

for scenario in scenario_list:

    # New conc df for updating with scenario data
    scen_conc_df = bsl_conc_df.copy()

    for t3_par in t3_par_li_no_ss:

        print(f"Scenario: {scenario}, T3 param: {t3_par}")

        # Extract series of reduction factors for this scenario and teotil3 par (index = real_river)
        river_factors = scen_summary_df_dict[(scenario, t3_par)].copy()
        river_factors = river_factors.loc[river_factors['Teotil3 parameter'] == t3_par, 'f-red_total']

        # Multiply baseline conc by the reduction factor
        for of800_par in var_mapping_teotil3_to_of800[t3_par]:
            scen_conc_df[of800_par] = bsl_conc_df[of800_par] * bsl_conc_df['real_river'].map(river_factors)

    # Bit of QC - check for NaNs
    if scen_conc_df.isna().sum().sum() > 0:
        print(f"{riv_no} has NaNs. Needs fixing!")

    scen_conc_dict[scenario] = scen_conc_df

In [ ]:
# Quick manual check for QC

riv = 7
t3_par = 'toc'
scen = 'Scenario_A'

# Baseline data
test_df = bsl_conc_df.copy()
test_df.loc[test_df['real_river'] == riv].head()

In [ ]:
# Factors
river_factors = scen_summary_df_dict[(scen, t3_par)].copy()
river_factors = river_factors.loc[river_factors['Teotil3 parameter'] == t3_par, 'f-red_total']
print(river_factors.loc[7].round(4))

In [ ]:
# Scenario data
test_df = scen_conc_dict[scen].copy()
test_df = test_df.loc[test_df['real_river'] == riv]
test_df.head()

## Generate netcdf files

Update the concentration columns for the variables that (may) have been updated through the scenarios.

In [ ]:
def update_ds_from_df(df: pd.DataFrame, ds_original: xr.Dataset,
                      coord_names=('real_river', 'river_time'),
                      history_string = None) -> xr.Dataset:
    """
    Create a copy of the original Dataset and update data variables for a subset of coordinates
    based on a modified DataFrame. Note: only currently works if cols in df vary over both coordinates in ds
    (e.g. doesn't work if latitude or longitude are in df).

    Parameters:
    - df: Modified pandas DataFrame (must include coord_names and updated data variables)
    - ds_original: Original xarray Dataset
    - coord_names: Tuple of coordinate names used to align updates
    Returns:
    - ds_updated: Dataset with updated values
    """

    # Copy original dataset, ready for updating
    ds_updated = ds_original.copy(deep=True)

    # Convert updated df to xarray dataset
    ds_patch = df.set_index(list(coord_names)).to_xarray()

    # Update variables in ds_updated that are present in df
    for var in ds_patch.data_vars:
        # Align and update only matching coordinates
        ds_updated[var].loc[ds_patch.coords] = ds_patch[var]

    # Optionally update the 'History' attribute of the new ds
    if history_string:
        existing_history = ds_original.attrs["history"]
        new_history = existing_history + history_string
        ds_updated.attrs['history'] = new_history

    return ds_updated


current_date = dt.datetime.now().date().strftime("%Y-%m-%d")

# Save the scenario ds for plotting below
ds_scen_dict = {}

for scenario in scenario_list:
    new_history_string = (
        f"; Update {current_date}, Leah JB (NIVA, ljb@niva.no): "
        f"Concentration reduction scenario {scenario}. v3: new scenarios as requested by MDir in 2025, including for more variables than just N. "
        "Compared to v2, also uses a simpler method to generate daily data from annual (assume constant daily source apportionment). "
        "See 'make_scenarios_simple_multi-vars.ipynb' in GitHub repository https://github.com/oslofjord-load-reductions/terrestrial-load-scenarios ")

    scen_ds = update_ds_from_df(scen_conc_dict[scenario], bsl_ds, history_string=new_history_string)

    # Truncate to start and end period, just to be on the safe side (other periods not updated)
    scen_ds = scen_ds.sel(river_time=slice(start_date, end_date))

    # Save
    netcdf_fpath = os.path.join(netcdf_outfolder, f"river-inputs_stage1_v{v}_{scenario}.nc")
    scen_ds.to_netcdf(netcdf_fpath)
    print(f"Scenario saved: {netcdf_fpath}")
    scen_ds.close()

    ds_scen_dict[scenario]= scen_ds

scen_ds

# Quick plot of results

Check the netcdf files look ok by doing a quick plot.

In [ ]:
plot_var = 'toc'
scen_summary_df_dict[('Scenario_B', plot_var)].head(8)

In [ ]:
var = "river_DOC"

ds_bsl = xr.open_dataset(fpath_baseline_data)
ds_bsl = ds_bsl.sel(river_time=slice(start_date, end_date))

for riv_no in oslofjord_riv_nos:
    riv_name = river_meta_df.loc[riv_no, "river_name"]
    fig, ax = plt.subplots(figsize=(10, 3))  # Create a new figure and axes for each river

    # Plot the baseline
    plot_bsl_ds = ds_bsl.sel(real_river=riv_no)
    ax.plot(plot_bsl_ds["river_time"].values, plot_bsl_ds[var].values, label="baseline")

    # Plot scenarios
    for scenario in scenario_list:
        plot_scen_ds = ds_scen_dict[scenario].copy().sel(real_river=riv_no)
        ax.plot(plot_scen_ds["river_time"].values, plot_scen_ds[var].values, label=scenario)

    ax.set_ylabel(var)
    ax.set_ylim(ymin=0)
    ax.legend()
    ax.set_title(f'{riv_no}: {riv_name}')
    plt.tight_layout()

    # fpath = os.path.join(fig_folder, "scenario_tseries", f"scenario_ts_{riv_name}_v{v}.png")
    # plt.savefig(fpath)
    plt.show()